In [3]:
# Complete Fine-tuning BERT for Sentiment Analysis in Google Colab
# Run this entire script in a single cell

# Step 1: Install required libraries
!pip install transformers datasets evaluate torch

# Step 2: Import all necessary libraries and DISABLE WANDB
import os
os.environ["WANDB_DISABLED"] = "true"  # ADD THIS LINE TO DISABLE WANDB

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate
import torch
import numpy as np
import pandas as pd

print("Libraries installed and imported successfully!")

# Step 3: Load and prepare dataset
print("Loading IMDB dataset...")
dataset = load_dataset("imdb")

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

# Tokenize datasets
print("Tokenizing dataset...")
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Prepare data for training
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets.set_format("torch")

# Use smaller subsets for quick training
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(500))

print(f"Training samples: {len(train_dataset)}")
print(f"Evaluation samples: {len(eval_dataset)}")

# Step 4: Load pre-trained model
print("Loading pre-trained BERT model...")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Step 5: Define evaluation metrics
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

# Step 6: Set training arguments with MULTIPLE wandb disabling methods
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    load_best_model_at_end=True,
    report_to=[],  # CHANGED: Use empty list instead of None
)

# Step 7: Initialize trainer
print("Initializing trainer...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

# Step 8: Start fine-tuning
print("Starting fine-tuning...")
trainer.train()

# Step 9: Evaluate the model
print("Evaluating model...")
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

# Step 10: Define prediction function
def predict_sentiment(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Ensure model is on the right device

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=256)
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Move tensors to device

    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)

    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_class = torch.argmax(probs, dim=-1).item()
    confidence = probs[0][predicted_class].item()

    sentiment = "positive" if predicted_class == 1 else "negative"
    return sentiment, confidence


# Step 11: Test predictions
print("\n" + "="*60)
print("TESTING FINE-TUNED MODEL")
print("="*60)

test_texts = [
    "This movie was absolutely fantastic! I loved every minute of it.",
    "Terrible film, waste of time and money.",
    "The movie was okay, nothing special but watchable.",
    "Outstanding performances and brilliant cinematography!",
    "Boring and predictable storyline."
]

for i, text in enumerate(test_texts, 1):
    sentiment, confidence = predict_sentiment(text)
    print(f"\nTest {i}:")
    print(f"Text: {text}")
    print(f"Predicted Sentiment: {sentiment.upper()}")
    print(f"Confidence: {confidence:.4f}")
    print("-" * 50)

# Step 12: Save the model (optional)
print("\nSaving fine-tuned model...")
model.save_pretrained("./fine-tuned-bert-sentiment")
tokenizer.save_pretrained("./fine-tuned-bert-sentiment")

print("\n" + "="*60)
print("FINE-TUNING COMPLETED SUCCESSFULLY!")
print("="*60)
print("Your model is ready to use for sentiment analysis!")


Libraries installed and imported successfully!
Loading IMDB dataset...
Tokenizing dataset...


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Training samples: 1000
Evaluation samples: 500
Loading pre-trained BERT model...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Initializing trainer...
Starting fine-tuning...


Epoch,Training Loss,Validation Loss,Accuracy
1,0.584800,0.313916,0.884000
2,0.323400,0.294054,0.880000


Evaluating model...


Evaluation results: {'eval_loss': 0.2940538227558136, 'eval_accuracy': 0.88, 'eval_runtime': 6.5421, 'eval_samples_per_second': 76.428, 'eval_steps_per_second': 4.891, 'epoch': 2.0}

TESTING FINE-TUNED MODEL

Test 1:
Text: This movie was absolutely fantastic! I loved every minute of it.
Predicted Sentiment: POSITIVE
Confidence: 0.8676
--------------------------------------------------

Test 2:
Text: Terrible film, waste of time and money.
Predicted Sentiment: NEGATIVE
Confidence: 0.9207
--------------------------------------------------

Test 3:
Text: The movie was okay, nothing special but watchable.
Predicted Sentiment: POSITIVE
Confidence: 0.6740
--------------------------------------------------

Test 4:
Text: Outstanding performances and brilliant cinematography!
Predicted Sentiment: POSITIVE
Confidence: 0.8732
--------------------------------------------------

Test 5:
Text: Boring and predictable storyline.
Predicted Sentiment: NEGATIVE
Confidence: 0.9180
-----------------------